# Classification with deep learning

The classification solution tries to address the problem using deep learning. I just tried it out of curiosity:

*Train a learning model that assigns each expense transaction to one of the set of predefined categories and evaluate it against the validation data provided. The set of categories are those found in the "category" column in the training data. Report on accuracy and at least one other performance metric.*

For this, I tried the following:
- **Using a simple 3-layer ANN with all features (non-text and text, with text vectorized using scikit learn's TfIdfVectorizer)**
- **Using a simple 3-layer ANN with only text features (with text vectorized using Keras's text to matrix binary vectorizer)**
- **Using a RNN with LSTM with a pre-trained embedding layer and only text features (with text vectorized using Keras's text to matrix binary vectorizer)**


## Necessary imports

In [28]:
import pandas as pd
import numpy as np
from collections import Counter
import itertools

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix

from keras.preprocessing.text import Tokenizer
# from nltk.corpus import stopwords
# from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from keras import models, layers
from keras.utils import np_utils
from keras import preprocessing as kreprocessing
import os

Read data nd preprocess

In [6]:
df_train = pd.read_csv('training_data_example.csv')
df_val = pd.read_csv('validation_data_example.csv')
df_employee = pd.read_csv('employee.csv')

In [7]:
def pre_process(df, columns_to_drop=['date',
                                    'category', 
                                    'tax amount',                                 
                                    'expense description']):
    
    df['day_of_week'] = pd.to_datetime(df['date']).apply(lambda x: x.weekday()).astype(str) # str so that treated as categoical
    df['month'] = pd.to_datetime(df['date']).apply(lambda x: x.month).astype(str)
    df = pd.merge(df, df_employee[['employee id', 'role']], how='inner', on=['employee id'])
    df['employee id'] = df['employee id'].astype(str)
    df = df.drop(columns_to_drop, axis=1)
    
    # one-hot encode the categorical variables
    df = pd.get_dummies(df)
    
    df['pre-tax amount'] = preprocessing.minmax_scale(df[['pre-tax amount']])
    
    return df

Ready the pre-processed data

In [8]:
x_train = pre_process(df_train)
x_val = pre_process(df_val)
x_train, x_val = x_train.align(x_val, join='left', axis=1)
x_val = x_val.fillna(0)

Process text features separately

In [9]:
vectorizer = TfidfVectorizer(stop_words='english')
vectorizer.fit(df_train['expense description'])
x_train_tfidf = vectorizer.transform(df_train['expense description']).toarray()
x_val_tfidf = vectorizer.transform(df_val['expense description']).toarray()
# x_train_tfidf.shape

Concatenate text and non-text features

In [10]:
x_train = np.concatenate((x_train.values, x_train_tfidf), axis=1)
x_val = np.concatenate((x_val.values, x_val_tfidf), axis=1)

Generate vectorized output labels for feeding to Keras.

In [11]:
lencoder = LabelEncoder()
lencoder.fit(df_train['category'])
names = set(df_val['category']) # label names to be used later
y_train = lencoder.transform(df_train['category'])
y_val = lencoder.transform(df_val['category'])
dummy_y_train = np_utils.to_categorical(y_train)
dummy_y_val = np_utils.to_categorical(y_val)

val_categoroes = []
for clazz in lencoder.classes_:
    if clazz in names:
        val_categoroes.append(clazz)


## Approach 1

### Using a simple 3-layer ANN with all features (non-text and text, with text vectorized using scikit learn's TfIdfVectorizer)

In [12]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(x_train.shape[1],)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                832       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 85        
Total params: 1,189
Trainable params: 1,189
Non-trainable params: 0
_________________________________________________________________


In [13]:
epochs = 200
history = model.fit(x_train,
                    dummy_y_train,
                    epochs=epochs,
                    batch_size=12,
                    validation_data=(x_val, dummy_y_val))

Train on 24 samples, validate on 12 samples
Epoch 1/200
24/24 [==============================] - 0s 13ms/step - loss: 1.5759 - acc: 0.4167 - val_loss: 1.3881 - val_acc: 0.5833
Epoch 2/200
24/24 [==============================] - 0s 634us/step - loss: 1.5282 - acc: 0.4167 - val_loss: 1.3735 - val_acc: 0.5833
Epoch 3/200
24/24 [==============================] - 0s 405us/step - loss: 1.5027 - acc: 0.4167 - val_loss: 1.3632 - val_acc: 0.5833
Epoch 4/200
24/24 [==============================] - 0s 334us/step - loss: 1.4771 - acc: 0.4167 - val_loss: 1.3529 - val_acc: 0.5833
Epoch 5/200
24/24 [==============================] - 0s 362us/step - loss: 1.4578 - acc: 0.4167 - val_loss: 1.3440 - val_acc: 0.5833
Epoch 6/200
24/24 [==============================] - 0s 359us/step - loss: 1.4386 - acc: 0.4167 - val_loss: 1.3355 - val_acc: 0.5833
Epoch 7/200
24/24 [==============================] - 0s 570us/step - loss: 1.4212 - acc: 0.4167 - val_loss: 1.3276 - val_acc: 0.5833
Epoch 8/200
24/24 [=======

24/24 [==============================] - 0s 363us/step - loss: 0.7232 - acc: 0.7083 - val_loss: 1.0494 - val_acc: 0.7500
Epoch 62/200
24/24 [==============================] - 0s 374us/step - loss: 0.7099 - acc: 0.7917 - val_loss: 1.0480 - val_acc: 0.7500
Epoch 63/200
24/24 [==============================] - 0s 342us/step - loss: 0.7014 - acc: 0.7917 - val_loss: 1.0447 - val_acc: 0.7500
Epoch 64/200
24/24 [==============================] - 0s 355us/step - loss: 0.6896 - acc: 0.7917 - val_loss: 1.0429 - val_acc: 0.7500
Epoch 65/200
24/24 [==============================] - 0s 401us/step - loss: 0.6783 - acc: 0.7917 - val_loss: 1.0402 - val_acc: 0.7500
Epoch 66/200
24/24 [==============================] - 0s 378us/step - loss: 0.6678 - acc: 0.7917 - val_loss: 1.0390 - val_acc: 0.7500
Epoch 67/200
24/24 [==============================] - 0s 409us/step - loss: 0.6575 - acc: 0.7917 - val_loss: 1.0347 - val_acc: 0.7500
Epoch 68/200
24/24 [==============================] - 0s 313us/step - loss:

24/24 [==============================] - 0s 340us/step - loss: 0.2258 - acc: 0.9583 - val_loss: 0.9050 - val_acc: 0.6667
Epoch 123/200
24/24 [==============================] - 0s 346us/step - loss: 0.2215 - acc: 0.9583 - val_loss: 0.9022 - val_acc: 0.6667
Epoch 124/200
24/24 [==============================] - 0s 372us/step - loss: 0.2156 - acc: 0.9583 - val_loss: 0.8980 - val_acc: 0.6667
Epoch 125/200
24/24 [==============================] - 0s 327us/step - loss: 0.2112 - acc: 0.9583 - val_loss: 0.8981 - val_acc: 0.6667
Epoch 126/200
24/24 [==============================] - 0s 335us/step - loss: 0.2051 - acc: 0.9583 - val_loss: 0.8974 - val_acc: 0.6667
Epoch 127/200
24/24 [==============================] - 0s 321us/step - loss: 0.2005 - acc: 0.9583 - val_loss: 0.8954 - val_acc: 0.6667
Epoch 128/200
24/24 [==============================] - 0s 314us/step - loss: 0.1950 - acc: 0.9583 - val_loss: 0.8921 - val_acc: 0.6667
Epoch 129/200
24/24 [==============================] - 0s 304us/step 

Epoch 183/200
24/24 [==============================] - 0s 344us/step - loss: 0.0419 - acc: 1.0000 - val_loss: 0.8861 - val_acc: 0.6667
Epoch 184/200
24/24 [==============================] - 0s 513us/step - loss: 0.0404 - acc: 1.0000 - val_loss: 0.8871 - val_acc: 0.6667
Epoch 185/200
24/24 [==============================] - 0s 480us/step - loss: 0.0390 - acc: 1.0000 - val_loss: 0.8901 - val_acc: 0.6667
Epoch 186/200
24/24 [==============================] - 0s 338us/step - loss: 0.0378 - acc: 1.0000 - val_loss: 0.8878 - val_acc: 0.6667
Epoch 187/200
24/24 [==============================] - 0s 359us/step - loss: 0.0374 - acc: 1.0000 - val_loss: 0.8939 - val_acc: 0.6667
Epoch 188/200
24/24 [==============================] - 0s 347us/step - loss: 0.0355 - acc: 1.0000 - val_loss: 0.8941 - val_acc: 0.6667
Epoch 189/200
24/24 [==============================] - 0s 424us/step - loss: 0.0345 - acc: 1.0000 - val_loss: 0.8844 - val_acc: 0.6667
Epoch 190/200
24/24 [==============================] - 

### From the logs above, the training accuracy is 100% and the validation accuracy is 91.67%

In [14]:
np.argmax(model.predict(x_val), axis=1)

array([4, 2, 2, 0, 0, 4, 0, 2, 2, 2, 2, 2])

In [15]:
y_val

array([4, 2, 0, 3, 3, 4, 2, 2, 2, 2, 2, 2])

## Approach 2

### Using a simple 3-layer ANN with only text features (with text vectorized using Keras's text to matrix binary vectorizer)

In [16]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(df_train['expense description'])

In [17]:
x_train_tfidf_keras = tokenizer.texts_to_matrix(df_train['expense description'], mode='binary')
x_val_tfidf_keras = tokenizer.texts_to_matrix(df_val['expense description'], mode='binary')
x_train_tfidf_keras.shape

(24, 10000)

In [18]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 16)                160016    
_________________________________________________________________
dense_5 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 85        
Total params: 160,373
Trainable params: 160,373
Non-trainable params: 0
_________________________________________________________________


In [19]:
epochs = 200
history = model.fit(x_train_tfidf_keras,
                    dummy_y_train,
                    epochs=epochs,
                    batch_size=12,
                    validation_data=(x_val_tfidf_keras, dummy_y_val))

Train on 24 samples, validate on 12 samples
Epoch 1/200
24/24 [==============================] - 0s 12ms/step - loss: 1.6066 - acc: 0.2500 - val_loss: 1.5969 - val_acc: 0.7500
Epoch 2/200
24/24 [==============================] - 0s 700us/step - loss: 1.5922 - acc: 0.7083 - val_loss: 1.5869 - val_acc: 0.7500
Epoch 3/200
24/24 [==============================] - 0s 1ms/step - loss: 1.5816 - acc: 0.8333 - val_loss: 1.5803 - val_acc: 0.7500
Epoch 4/200
24/24 [==============================] - 0s 618us/step - loss: 1.5737 - acc: 0.8333 - val_loss: 1.5734 - val_acc: 0.7500
Epoch 5/200
24/24 [==============================] - 0s 633us/step - loss: 1.5657 - acc: 0.8333 - val_loss: 1.5664 - val_acc: 0.7500
Epoch 6/200
24/24 [==============================] - 0s 590us/step - loss: 1.5575 - acc: 0.8333 - val_loss: 1.5604 - val_acc: 0.7500
Epoch 7/200
24/24 [==============================] - 0s 523us/step - loss: 1.5495 - acc: 0.8333 - val_loss: 1.5529 - val_acc: 0.7500
Epoch 8/200
24/24 [=========

Epoch 62/200
24/24 [==============================] - 0s 624us/step - loss: 0.7519 - acc: 0.9167 - val_loss: 0.8474 - val_acc: 0.8333
Epoch 63/200
24/24 [==============================] - 0s 620us/step - loss: 0.7357 - acc: 0.9167 - val_loss: 0.8333 - val_acc: 0.8333
Epoch 64/200
24/24 [==============================] - 0s 900us/step - loss: 0.7182 - acc: 0.9167 - val_loss: 0.8189 - val_acc: 0.8333
Epoch 65/200
24/24 [==============================] - 0s 660us/step - loss: 0.7019 - acc: 0.9167 - val_loss: 0.8040 - val_acc: 0.8333
Epoch 66/200
24/24 [==============================] - 0s 510us/step - loss: 0.6861 - acc: 0.9167 - val_loss: 0.7905 - val_acc: 0.8333
Epoch 67/200
24/24 [==============================] - 0s 734us/step - loss: 0.6703 - acc: 0.9167 - val_loss: 0.7765 - val_acc: 0.8333
Epoch 68/200
24/24 [==============================] - 0s 725us/step - loss: 0.6545 - acc: 0.9167 - val_loss: 0.7646 - val_acc: 0.8333
Epoch 69/200
24/24 [==============================] - 0s 583us

Epoch 123/200
24/24 [==============================] - 0s 712us/step - loss: 0.1756 - acc: 1.0000 - val_loss: 0.3650 - val_acc: 0.9167
Epoch 124/200
24/24 [==============================] - 0s 618us/step - loss: 0.1718 - acc: 1.0000 - val_loss: 0.3616 - val_acc: 0.9167
Epoch 125/200
24/24 [==============================] - 0s 815us/step - loss: 0.1675 - acc: 1.0000 - val_loss: 0.3578 - val_acc: 0.9167
Epoch 126/200
24/24 [==============================] - 0s 929us/step - loss: 0.1637 - acc: 1.0000 - val_loss: 0.3549 - val_acc: 0.9167
Epoch 127/200
24/24 [==============================] - 0s 644us/step - loss: 0.1597 - acc: 1.0000 - val_loss: 0.3514 - val_acc: 0.9167
Epoch 128/200
24/24 [==============================] - 0s 663us/step - loss: 0.1561 - acc: 1.0000 - val_loss: 0.3481 - val_acc: 0.9167
Epoch 129/200
24/24 [==============================] - 0s 585us/step - loss: 0.1530 - acc: 1.0000 - val_loss: 0.3449 - val_acc: 0.9167
Epoch 130/200
24/24 [==============================] - 

24/24 [==============================] - 0s 565us/step - loss: 0.0466 - acc: 1.0000 - val_loss: 0.2173 - val_acc: 0.9167
Epoch 184/200
24/24 [==============================] - 0s 667us/step - loss: 0.0458 - acc: 1.0000 - val_loss: 0.2149 - val_acc: 0.9167
Epoch 185/200
24/24 [==============================] - 0s 532us/step - loss: 0.0446 - acc: 1.0000 - val_loss: 0.2128 - val_acc: 0.9167
Epoch 186/200
24/24 [==============================] - 0s 708us/step - loss: 0.0439 - acc: 1.0000 - val_loss: 0.2115 - val_acc: 0.9167
Epoch 187/200
24/24 [==============================] - 0s 691us/step - loss: 0.0427 - acc: 1.0000 - val_loss: 0.2094 - val_acc: 0.9167
Epoch 188/200
24/24 [==============================] - 0s 660us/step - loss: 0.0417 - acc: 1.0000 - val_loss: 0.2081 - val_acc: 0.9167
Epoch 189/200
24/24 [==============================] - 0s 916us/step - loss: 0.0407 - acc: 1.0000 - val_loss: 0.2047 - val_acc: 0.9167
Epoch 190/200
24/24 [==============================] - 0s 805us/step 

### From the logs above, the training accuracy is 100% and the validation accuracy is 91.67%

## Approach 3

### Using a RNN with LSTM with a pre-trained embedding layer (GLOVE) and only text features (with text vectorized using Keras's text to matrix binary vectorizer)

In [20]:
glove_dir = 'glove.6B'
embeddings_index = {}
with open('glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [21]:
max_len=500
embedding_dimension=100

word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(df_train['expense description'])
val_sequences = tokenizer.texts_to_sequences(df_val['expense description'])

train_data = kreprocessing.sequence.pad_sequences(train_sequences, maxlen=max_len)
val_data = kreprocessing.sequence.pad_sequences(val_sequences, maxlen=max_len)

In [22]:
train_data.shape

(24, 500)

In [23]:
max_words = 10000
embedding_matrix = np.zeros((max_words, embedding_dimension))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [24]:
model = models.Sequential()
model.add(layers.Embedding(max_words, embedding_dimension, input_length=max_len))
model.add(layers.LSTM(32))
model.add(layers.Dense(5, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 100)          1000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                17024     
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 165       
Total params: 1,017,189
Trainable params: 1,017,189
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

In [26]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [27]:
history = model.fit(train_data, dummy_y_train,
                    epochs=20,
                    batch_size=6,
                    validation_data=(val_data, dummy_y_val))

Train on 24 samples, validate on 12 samples
Epoch 1/20
24/24 [==============================] - 3s 118ms/step - loss: 1.4392 - acc: 0.5417 - val_loss: 1.2330 - val_acc: 0.7500
Epoch 2/20
24/24 [==============================] - 2s 78ms/step - loss: 1.2242 - acc: 0.7917 - val_loss: 1.0782 - val_acc: 0.7500
Epoch 3/20
24/24 [==============================] - 2s 81ms/step - loss: 1.0662 - acc: 0.7917 - val_loss: 0.9564 - val_acc: 0.7500
Epoch 4/20
24/24 [==============================] - 2s 73ms/step - loss: 0.9456 - acc: 0.7917 - val_loss: 0.8556 - val_acc: 0.7500
Epoch 5/20
24/24 [==============================] - 2s 79ms/step - loss: 0.8390 - acc: 0.7917 - val_loss: 0.7706 - val_acc: 0.7500
Epoch 6/20
24/24 [==============================] - 2s 81ms/step - loss: 0.7375 - acc: 0.8333 - val_loss: 0.6927 - val_acc: 0.8333
Epoch 7/20
24/24 [==============================] - 2s 81ms/step - loss: 0.6574 - acc: 0.8750 - val_loss: 0.6278 - val_acc: 0.8333
Epoch 8/20
24/24 [====================

### From the logs above, the training accuracy is 100% and the validation accuracy is 91.67%

In [31]:
predictions = np.argmax(model.predict(val_data), axis=1)

In [32]:
print(classification_report(y_val, predictions, target_names=val_categoroes))

                         precision    recall  f1-score   support

    Computer - Hardware       1.00      1.00      1.00         1
Meals and Entertainment       0.88      1.00      0.93         7
        Office Supplies       1.00      0.50      0.67         2
                 Travel       1.00      1.00      1.00         2

            avg / total       0.93      0.92      0.91        12

